## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

User number u243700
Enter Password:  ··············


<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [5]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [6]:
df_lounge_eligibility.head(2)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13


In [7]:
df_lounge_eligibility.shape

(1534750, 12)

In [8]:
# a look to a particular flight number and date

#mask = (df_lounge_eligibility['DISCHARGE_STN_CD'] == 'GCM   ') & (df_lounge_eligibility['GMT_UPLIFT_DT'] == dt.date(2019,9,12) )

#df_lounge_eligibility[mask]

#### S19 Flight info

In [9]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [10]:
df_flight_info.head(2)
df_flight_info.dtypes

GMT_PLND_DEP_TS    datetime64[ns]
GMT_ACT_DEP_TS     datetime64[ns]
OPG_ALN_CD                 object
OPG_FLT_NO                  int64
ACT_DEP_STN_CD             object
ACT_DEP_TML_CD             object
PLND_ARR_STN_CD            object
ACT_ARR_STN_CD             object
IATA_AC_TYP_CD             object
ACT_AC_TYP_CD              object
ROUTE                      object
dtype: object

#### Station Code Decode

In [11]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [12]:
df_country.head(2)

,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


#### Additional Station Decodes from ICW reference table

In [13]:
# There are lots of missing destinations which are in lounge_elig but not in df_country.
# They are all new (since ~2019) arrivals to Heathrow.
# They are not in sandbox country dataset, but they are in a reference table on ICW.
# Load that ICW reference table and filter for results not in our merged table, but that are in lounge_elig.
# This should result in zero nan values.
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
"""
df_additional_country_decodes=icw.queryToDataframe(query)
df_additional_country_decodes.shape # we will remove destinations already in df_country and not in df_lounge_elig later.

(10975, 5)

In [14]:
df_additional_country_decodes.head(5)

,STN_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,RFS,NI,Nicaragua,CENTRAL AMERICA,SOUTH AMERICA INC CARIBBEAN
1,COO,BJ,Benin,WEST AFRICA,AFRICA
2,SFY,US,United States,USA,NORTH AMERICA
3,HGO,CI,Cote d'Ivoire,WEST AFRICA,AFRICA
4,RKA,PF,French Polynesia,PACIFIC ISLANDS,AUSTRALASIA PACIFIC


#### Aircraft Type

In [15]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [16]:
df_acft_typ.head(2)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235


In [17]:
df_acft_typ.shape

(35, 7)

In [18]:
df_acft_typ.head(5)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165


<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [19]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [20]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

In [21]:
# Note that some columns have blank spaces!
format_df
format_2

{'GMT_PLND_DEP_TS': Timestamp('2023-04-30 20:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-04-30 20:34:00'),
 'OPG_ALN_CD': 'BA    ',
 'OPG_FLT_NO': 57,
 'ACT_DEP_STN_CD': 'LHR   ',
 'ACT_DEP_TML_CD': '5 ',
 'PLND_ARR_STN_CD': 'JNB   ',
 'ACT_ARR_STN_CD': 'JNB   ',
 'IATA_AC_TYP_CD': '388   ',
 'ACT_AC_TYP_CD': 'A8  ',
 'ROUTE': 'LHRJNB      '}

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [22]:
# Stripping strings

tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_additional_country_decodes]

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

In [23]:
# Removing duplucates of flight_info

# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")


In [24]:
# 2. Flight info
# Ian's code to clean dataset - identify latest departure date for
# duplicated flights and eliminate it.
df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

In [25]:
# 3. Country
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)


In [26]:
# 4. Aircraft type
df_acft_typ.sort_values('FIRST_SEATS_QTY').tail(10)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
8,789,L8,WB,8,42,39,127
21,77W,F7,WB,8,76,40,130
34,777,R7,WB,10,52,40,134
29,777,W7,WB,12,48,32,127
12,77W,G7,WB,14,56,44,183
13,744,S4,WB,14,86,30,145
22,744,V4,WB,14,52,36,235
15,388,A8,WB,14,97,55,303
27,777,V7,WB,14,48,40,124
24,777,P7,WB,17,48,24,127


In [27]:
df_lounge_eligibility.shape

(1534750, 12)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [28]:
# df_lounge_eligibility.head(5)

In [29]:
df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00


In [30]:
# your code here!
df_lounge_elig_flight_info.shape

(1534750, 25)

Get rows with any null values

In [31]:
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].shape)
# There are currently 1183 rows for which flights are not available.

(1183, 25)


In [32]:
# Example of missing flight number.
print(df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642].head(5))
print(df_lounge_elig_flight_info[df_lounge_elig_flight_info['OPERATING_FLT_NO']==8642]['Skew_Id'].head(5))

Empty DataFrame
Columns: [GMT_PLND_DEP_TS, GMT_ACT_DEP_TS, OPG_ALN_CD, OPG_FLT_NO, ACT_DEP_STN_CD, ACT_DEP_TML_CD, PLND_ARR_STN_CD, ACT_ARR_STN_CD, IATA_AC_TYP_CD, ACT_AC_TYP_CD, ROUTE, GMT_PLND_DEP_DT, GMT_PLND_DEP_TIME]
Index: []
300      2019-09-148642
876      2019-09-148642
1452     2019-09-148642
16459    2019-06-158642
17035    2019-06-158642
Name: Skew_Id, dtype: object


### Merge `df_country` to table

In [33]:
[headers_and_first_row(d) for d in [df_lounge_elig_flight_info,df_country]]

[{'Skew_Id': '2023-08-05618',
  'OPERATING_AIRLINE_CD': 'BA',
  'OPERATING_FLT_NO': 618,
  'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
  'UPLIFT_STN_CD': 'LHR',
  'DISCHARGE_STN_CD': 'OLB',
  'BOOKED_CABIN_CD': 'C',
  'TRAVEL_CABIN_CD': 'C',
  'BA_PAX_TIER': 'Gold For Life',
  'ONEWORLD_TIER': 'EMER',
  'Lounge_eligibility_tier': 'Tier 2',
  'pax': 2,
  'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
  'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
  'OPG_ALN_CD': 'BA',
  'OPG_FLT_NO': 618.0,
  'ACT_DEP_STN_CD': 'LHR',
  'ACT_DEP_TML_CD': '3',
  'PLND_ARR_STN_CD': 'OLB',
  'ACT_ARR_STN_CD': 'OLB',
  'IATA_AC_TYP_CD': '319',
  'ACT_AC_TYP_CD': 'M4',
  'ROUTE': 'LHROLB',
  'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
  'GMT_PLND_DEP_TIME': datetime.time(14, 25)},
 {'ROUTE': 'LHRINV',
  'COUNTRY_CD': 'GB',
  'COUNTRY_NM': 'United Kingdom and Northern Ireland',
  'CORP_GEOG_CTRY_GRP_NM': 'UK',
  'CORP_GEOG_CONTINENT_NM': 'UK',
  'DEP_STN_CD': 'LHR',
  'ARR_STN_CD': 'INV'}]

In [34]:
df_additional_country_decodes.columns
df_country.columns

Index(['ROUTE', 'COUNTRY_CD', 'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM',
       'CORP_GEOG_CONTINENT_NM', 'DEP_STN_CD', 'ARR_STN_CD'],
      dtype='object')

In [35]:
# For efficiency, we should merge df_country and df_additional_country_decodes first.
# Firstly, organise df_additional_country_decodes to have the same columns.
# if statement to avoid throwing errors if column renaming from cell below has already been done.
if 'STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'STN_CD'
elif 'ARR_STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'ARR_STN_CD'
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes[station_code_col].isin(df_country['ARR_STN_CD'])]
# for more efficiency, we can remove all those rows which are not needed as there are no lounge elig rows with that destinations.
df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes[station_code_col].isin(df_lounge_elig_flight_info['DISCHARGE_STN_CD'])]
print(df_additional_country_decodes.shape)

(30, 5)


In [36]:
# We need df_additional_country_decodes to have the same column names and order as df_country, ready for pd.concat.
# By merging now, we eliminate the need to merge separately with df_lounge_elig.
df_additional_country_decodes['ROUTE'] = 'LHR' + df_additional_country_decodes['STN_CD']
df_additional_country_decodes['DEP_STN_CD'] = 'LHR'
# We must ensure column names and order of df_additional_country_decodes match df_country.
df_additional_country_decodes.rename(columns={'STN_CD':'ARR_STN_CD'},inplace=True)
df_additional_country_decodes = df_additional_country_decodes[df_country.columns]

In [37]:
# concat together.
df_country = pd.concat([df_country,df_additional_country_decodes])

In [38]:
df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['DISCHARGE_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI


In [39]:
df_lounge_elig_flight_info_country.shape

(1534750, 32)

Check for null values

In [40]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['CORP_GEOG_CONTINENT_NM'].isna()].head(5)
# There are no null values arising from lack of country data.

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


### `df_acft_typ`

In [41]:
df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0


In [42]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1534750, 37)

Check for null

In [43]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
300,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
466,2019-09-298598,BA,8598,2019-09-29,LHR,CFU,M,M,,,Not eligible,43,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRCFU,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,CFU,NaN,NaN,NaN,NaN,NaN
876,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
1452,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,,,Not eligible,144,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
11711,2019-06-168586,BA,8586,2019-06-16,LHR,KLX,M,M,Gold,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRKLX,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,KLX,NaN,NaN,NaN,NaN,NaN


In [46]:
df_acft_typ[df_acft_typ['ACT_AC_TYP_CD'] == 'M4']

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
20,319,M4,NB,0,21,0,113


### More cleaning for columns redundancy

In [45]:
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

{'Skew_Id': '2023-08-05618',
 'OPERATING_AIRLINE_CD': 'BA',
 'OPERATING_FLT_NO': 618,
 'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
 'UPLIFT_STN_CD': 'LHR',
 'DISCHARGE_STN_CD': 'OLB',
 'BOOKED_CABIN_CD': 'C',
 'TRAVEL_CABIN_CD': 'C',
 'BA_PAX_TIER': 'Gold For Life',
 'ONEWORLD_TIER': 'EMER',
 'Lounge_eligibility_tier': 'Tier 2',
 'pax': 2,
 'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
 'OPG_ALN_CD': 'BA',
 'OPG_FLT_NO': 618.0,
 'ACT_DEP_STN_CD': 'LHR',
 'ACT_DEP_TML_CD': '3',
 'PLND_ARR_STN_CD': 'OLB',
 'ACT_ARR_STN_CD': 'OLB',
 'IATA_AC_TYP_CD': '319',
 'ACT_AC_TYP_CD': 'M4',
 'ROUTE_x': 'LHROLB',
 'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
 'GMT_PLND_DEP_TIME': datetime.time(14, 25),
 'ROUTE_y': 'LHROLB',
 'COUNTRY_CD': 'IT',
 'COUNTRY_NM': 'Italy',
 'CORP_GEOG_CTRY_GRP_NM': 'MEDITERRANEAN',
 'CORP_GEOG_CONTINENT_NM': 'EUROPE EXC UK',
 'DEP_STN_CD': 'LHR',
 'ARR_STN_CD': 'OLB',
 'WB_NB_CAT': 'NB',
 'FIRST_SEATS_QTY': 0.0

In [49]:
# drop redundant columns.

df_final = df_lounge_elig_flight_info_country_acft_typ.copy()

to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','GMT_UPLIFT_DT','ROUTE_x',
          'GMT_PLND_DEP_DT','GMT_PLND_DEP_TIME','ROUTE_y','DEP_STN_CD','ARR_STN_CD']

df_final = df_final.drop(columns = to_drop,)

df_final.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,ACT_DEP_TML_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,3,319,M4,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,LHR,STR,C,C,,,Tier 3,13,3,319,M4,DE,Germany,WEST EUROPE,EUROPE EXC UK,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,LHR,OSL,M,M,,,Not eligible,75,5,32A,H3,NO,Norway,SCANDINAVIA,EUROPE EXC UK,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,LHR,NCE,M,M,,EMER,Tier 2,6,5,319,A4,FR,France,WEST EUROPE,EUROPE EXC UK,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,LHR,CAI,J,J,,,Tier 3,23,5,343,W9,EG,Egypt,NORTH AFRICA,AFRICA,WB,0.0,45.0,0.0,212.0


Save as CSV

In [50]:
df_final.to_csv('scragg_merged_df_final_2023-10-31-1330.csv')

<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [ ]:
# your code here!

In [ ]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_wb_nb_ttl

In [ ]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [ ]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
# a look at the data
df_groupby_wb_nb

In [ ]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [ ]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [ ]:
# final table
df_groupby_wb_nb

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [ ]:
# your code here!

In [ ]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [ ]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

In [ ]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [ ]:
# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
# a look at the data
df_groupby_flt_no.head(4)

In [ ]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [ ]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [ ]:
# final table
df_groupby_flt_no.head(4)

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

### Ian  
First of all I would like to be able to split the data into 2019/23 data separately, which can be done by adding a year field into the table.

In [ ]:
# Original merged table
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

In [ ]:
# df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEPT_YR'] = df_lounge_elig_flight_info_country_acft_typ['GMT_PLND_DEP_DT'].year

# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)
df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_YR'] = [dts.year for dts in df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_DT']]

df_lounge_elig_flight_info_country_acft_typ.head(10)

In [ ]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year

### Linear Regression

In [ ]:
# import numpy as np
# from sklearn import datasets, linear_model
# import matplotlib.pyplot as plt

In [ ]:
# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

Separate the large tables to different tiers

In [ ]:
# df_merged_T1 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 1'].reset_index()
# df_merged_T2 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 2'].reset_index()
# df_merged_T3 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 3'].reset_index()

# df_merged_T1 = df_merged_T1.rename(columns={'pax':'Tier_1_pax'})
# df_merged_T2 = df_merged_T2.rename(columns={'pax':'Tier_2_pax'})
# df_merged_T3 = df_merged_T3.rename(columns={'pax':'Tier_3_pax'})

In [ ]:
# df_merged_T3.columns
# headers_and_first_row(df_merged_T1)


In [ ]:
# y = np.array([1,2,3])
# x = np.array([-1,0,1])

# # y = y.to_numpy()
# # x = x.to_numpy().reshape(-1, 1)
# x = x.reshape(-1,1)

# # print(x.shape, y.shape)

# regr = linear_model.LinearRegression()
# regr.fit(x,y)

# regr.coef_, regr.intercept_, regr.rank_

### Compare by Year

In [ ]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year_ttl

In [ ]:
df_groupby_year = pd.merge(df_groupby_year,
                            df_groupby_year_ttl,
                            on = ['GMT_UPLIFT_YR'],
                            how = 'left'
                           )

df_groupby_year

In [ ]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_year['pax_eligible%'] = (df_groupby_year['pax_count'] / df_groupby_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_year['pax_eligible%'] = df_groupby_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [ ]:
df_groupby_year = df_groupby_year[df_groupby_year['Lounge_eligibility_tier'] != "Not eligible"]

df_groupby_year

Not much difference

### Compare by location and year

In [ ]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_country_group_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR']

df_groupby_country_group_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_country_group_year = pd.merge(df_groupby_country_group_year,
                                         df_groupby_country_group_year_ttl,
                                         on = list_groupby,
                                         how = "left")

df_groupby_country_group_year = df_groupby_country_group_year[df_groupby_country_group_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_country_group_year['pax_eligible%'] = (df_groupby_country_group_year['pax_count'] / df_groupby_country_group_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_country_group_year['pax_eligible%'] = df_groupby_country_group_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_country_group_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_country_group_year.head(6)


In [ ]:
df_groupby_country_group_year = df_groupby_country_group_year.set_index(['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_country_group_year

### By continent

In [ ]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_cont_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR']

df_groupby_cont_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_cont_year = pd.merge(df_groupby_cont_year,
                                df_groupby_cont_year_ttl,
                                on = list_groupby,
                                how = "left")

df_groupby_cont_year = df_groupby_cont_year[df_groupby_cont_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_cont_year['pax_eligible%'] = (df_groupby_cont_year['pax_count'] / df_groupby_cont_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_cont_year['pax_eligible%'] = df_groupby_cont_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_cont_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_cont_year.head(6)

In [ ]:
df_groupby_cont_year = df_groupby_cont_year.set_index(['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_cont_year

### By aircraft

In [ ]:
list_groupby = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_acft = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                mean_pax_count = ('pax','mean')).reset_index()

df_groupby_acft['mean_pax_count'] = df_groupby_acft['mean_pax_count'].map('{:,.1f}'.format)

df_groupby_acft = df_groupby_acft[df_groupby_acft['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_acft = df_groupby_acft.set_index(list_groupby,drop = True).unstack('Lounge_eligibility_tier')

df_groupby_acft.fillna(0, inplace=True)

df_groupby_acft


<br><br><br>

In [ ]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [ ]:
# your code here

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.